In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-10-20


Found rundate 2024-10-19 10:00:04 < 2024-10-20 at commit 22261ee2; returning commit aa77ea3


'2024-10-20 10:00:04'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-10-20 10:00:04, NJSP was reporting 521 YTD deaths
Current YTD Deaths (2025-10-20 10:00:01-04:00): 463
Previous year YTD Deaths (adjusted; 2024-10-20 10:00:04): 520.6465861664334
Projected 2025 total: 611.8654894407318
-11.1% change


8d5d1a33f1f603bc724d9537617a6d4365babc33: FAUQStats cache miss: 2024, Sun Oct 20 10:00:04 EDT 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,HIGHWAY,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,INJURIES,STREET,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,1,State Highway 1,1.0,0.0,0.0,1.0,0.0,4.0,NaN,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,129,State Highway 129,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,446,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,NaN,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,East Oak St,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,647,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13491,05,Cape May,0504,Dennis Twsp,47,State Highway 47 MP 18.8,1.0,1.0,0.0,0.0,0.0,1.0,NaN,2024-10-15 12:28:00-04:00
13492,03,Burlington,0338,Westampton Twsp,95,New Jersey Turnpike MP 43.9,1.0,1.0,0.0,0.0,0.0,0.0,NaN,2024-10-15 21:56:00-04:00
13494,03,Burlington,0313,Evesham Twsp,73,State Highway 73,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2024-10-16 12:33:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,17,3,7,0,26
Bergen,19,1,4,0,24
Burlington,20,5,7,0,30
Camden,18,2,12,2,33
Cape May,7,3,0,1,10
Cumberland,9,2,6,0,16
Essex,11,3,23,1,38
Gloucester,16,4,3,0,20
Hudson,7,1,4,1,11


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

80.1% through the year, 19.9% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,24,32,17
Bergen,driver,14,14,19
Burlington,driver,26,31,20
Camden,driver,14,19,18
Cape May,driver,6,8,7
...,...,...,...,...
Salem,crashes,12,14,15
Somerset,crashes,14,18,10
Sussex,crashes,11,12,10


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,36,0,23,3,11
Bergen,30,0,19,1,7
Burlington,38,0,24,5,12
Camden,41,2,24,3,13
Cape May,15,1,9,4,0
Cumberland,20,0,10,2,11
Essex,49,1,14,8,30
Gloucester,32,0,22,10,6
Hudson,18,1,9,1,10


In [9]:
projected.to_csv(PROJECTED_CSV)